In [1]:
import numpy as np
x = np.random.normal(0, 1, 100_000_000)

In [2]:
def py_count(array, threshold):
    counter = 0
    for entry in array:
        if entry > threshold:
            counter += 1
            
    return counter

In [11]:
%timeit py_count(x[:10_000_000], 4)

2.2 s ± 17.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
def np_count(array, threshold):
    above_threshold = array > threshold
    return np.count_nonzero(above_threshold)

In [12]:
%timeit np_count(x[:10_000_000], 4)

7.25 ms ± 186 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
print("""

extern "C"
int c_count(int len, double* array, double threshold)
{
    int i;
    int counter = 0;
    
    for (i = 0; i < len; ++i)
        if (array[i] > threshold)
            counter ++;
            
    return counter;
}
""", file=open("source.C", "w"))

!gcc source.C -o mylib.so -fPIC --shared

In [20]:
!gcc --version

gcc (Ubuntu 11.2.0-19ubuntu1) 11.2.0
Copyright (C) 2021 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [16]:
!ls

my-first-file.txt  SOSC22-livesessions	storage_ex.ipynb   Untitled.ipynb
mylib.so	   source.C		test-download.txt


In [17]:
import ctypes

mylib = ctypes.CDLL("./mylib.so")
mylib.c_count

In [18]:
mylib.c_count.restype = ctypes.c_int
mylib.c_count.argtypes = [
    ctypes.c_int,
    np.ctypeslib.ndpointer(np.float64),
    ctypes.c_double
]

In [21]:
%timeit mylib.c_count(len(x), x, 4)

249 ms ± 531 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%timeit np_count(x, 4)

89.7 ms ± 8.43 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
